In [1]:
import time
from context import remove_far_waters
import mdtraj as md
import numpy as np

In [2]:
sample_trj = 'data/bc_1.xtc'
sample_ref = 'data/md_ref.pdb'

In [3]:
def timer(func_):
    '''Decorator for timing functions.'''
    def function_timer(*args, **kwargs):
        start = time.time()
        result = func_(*args, **kwargs)
        stop = time.time()
        runtime = stop - start
        print(f'Completed {func_.__name__} in runtime of {runtime:10.2f} seconds.')
        
        return result
    return function_timer

In [4]:
t = md.load(sample_trj, top=sample_ref)
print(f'Number of Frames: \t {t.n_frames}')
print(f'Number of Atoms: \t {t.n_atoms}')

Number of Frames: 	 1001
Number of Atoms: 	 40833


### Remove Far Waters

In [34]:
@timer
def init_removewaters():
    t = md.load(sample_trj, top=sample_ref)

    a = remove_far_waters.RemoveWaters(t, sel_query='resid 237', sel='protein', n_waters=100, cutoff=1., verbose=True)
    
init_removewaters()

Trajectory with 1001 frames and 40833 atoms.
36705 water atoms in trajectory.
Completed init_removewaters in runtime of       3.28 seconds.


In [35]:
@timer
def static_search():
    t = md.load(sample_trj, top=sample_ref)
    a = remove_far_waters.RemoveWaters(t, sel_query='resid 237', sel='protein', n_waters=100, cutoff=1., verbose=True)
    t_new = a.static_search()
    
static_search()

Trajectory with 1001 frames and 40833 atoms.
36705 water atoms in trajectory.
Found 47 water molecules within 1.0 nm.
Completed static_search in runtime of       3.69 seconds.


In [5]:
@timer
def dynamic_search():
    t = md.load(sample_trj, top=sample_ref)
    a = remove_far_waters.RemoveWaters(t, sel_query='resid 237', sel='protein', n_waters=100, cutoff=0.5, verbose=True)
    t_new = a.dynamic_search()
    verify = remove_far_waters.verify(t, t_new, 'trj_dynamic.txt', errors=True, verbose=True)
    if verify:
        print('verification complete.')

dynamic_search()

Trajectory with 1001 frames and 40833 atoms.
36705 water atoms in trajectory.
Distance calculation completed.
Found a maximum of 55 water molecules within a cutoff of 0.5 nm.
verification complete.
Completed dynamic_search in runtime of      87.41 seconds.


In [6]:
@timer
def dynamic_zero():
    t = md.load(sample_trj, top=sample_ref)
    a = remove_far_waters.RemoveWaters(t, sel_query='resid 237', sel='protein', n_waters=10, cutoff=0.5)
    t_new = a.dynamic_zero()

dynamic_zero()

Completed dynamic_zero in runtime of       6.02 seconds.
